In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import time
from scipy.stats import linregress
import os
import csv

# Import API key
from api_keys import api_key_file

# Output File (CSV)
output_data_file = "output_data/duistats.csv"

# File to Load (**Remember to Change These file names to DUI saved file names)
2013_dui_data = "wherever file is saved/nrs_2013_combined.csv"
2007_dui_data = "wherever file is saved/nrs_2007_final.csv"

# Read DUI 2007 and 2013 Data Files and store into Pandas DataFrames(can't start with a number)
dui2013_data = pd.read_csv(2013_dui_data)
dui2007_data = pd.read_csv(2007_dui_data)

#renames for merge
dui2007_data.rename(columns = {"STATE": "State"}, inplace = True)

# Combine the data into a single dataset.  
dui_data_complete = pd.merge(dui2013_data,dui2007_data, how="left", on=["State"])

In [ ]:
# Create an overview table grouped by State
per_state = dui_data_complete.set_index("State").groupby(["State"])
per_state.head()

In [ ]:
#Declare columns/variables to keep and create a summary dataframe
#links to data dictionaries for dui studies (help decided relevant columns)
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/2007_nrsdatadictionary.pdf
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/nrs_2013datadictionary.pdf

timeday
STATE
NRS_AREA
DATE

dui_summary = pd.DataFrame({})

dui_summary.head()

In [ ]:
#census data link to retrieve population information per state
#https://www.census.gov/quickfacts/fact/table/US/PST040219